# Use duckdb to read from postgis for area

Do some testing with DuckDB for spatial analysis. 

We have postgis which can handle spatial queries. At the moment we use postgis as postgres and has not enabled the spatial datatypes.

We plan is to move some history data to parquet and need a query language that works across parquet and postgres/postgis


Creating geojson using a map can be done from <https://geojson.io>

In [ ]:
from pathlib import Path
import duckdb
import json
from shapely.geometry import shape

We need to setup DuckDB and install/load spatial and postgres support. 

We use an in memory DuckDB database since all data is external to DuckDB.

In [ ]:
con = duckdb.connect()
for ext in ["postgres", "spatial"]:
    con.install_extension(ext)
    con.load_extension(ext)
con.sql("""
    ATTACH 'dbname=spartid_ais user=postgres password=postgres host=127.0.0.1' AS db (TYPE postgres); 
""")

We define some geojson polygons in files and use that to define areas of interest.

In [ ]:
within_area_file = Path("./inner_oslofjord.geojson")
# within_area_file = Path("./mandal_to_bergen.geojson")
feature = json.loads(within_area_file.read_text())
feature

In [ ]:
area = shape(feature["features"][0]["geometry"])
area

## Query last ship positions

In [ ]:
con.sql("SELECT COUNT(*) FROM db.public.last_position")

In [ ]:
# Execute the SQL query to 
query = """
SELECT *, ST_POINT(long, lat) as geom, datediff('hour', now() AT TIME ZONE 'UTC', timestamp)
FROM db.public.last_position
WHERE ST_INTERSECTS(geom, (?)) AND datediff('hour', now()::timestamp, timestamp) > -6
"""

con.execute(query, (area.wkt,)).df()

In [ ]:
" AND ".join([f"ST_INTERSECTS(geom, (?))" for x in (1, 2, 3)])

## Query historic positions

In [ ]:
con.sql("SELECT COUNT(*) FROM db.public.historic_position")

In [ ]:
# Execute the SQL query to 
query = """
SELECT *, ST_POINT(long, lat) as geom, datediff('hour', now() AT TIME ZONE 'UTC', timestamp)
FROM db.public.historic_position
WHERE ST_WITHIN(geom, (?)) AND datediff('days', now()::timestamp, timestamp) > -6
"""

con.execute(query, (area.wkt,)).df()

In [ ]:
areas = [shape(x["geometry"]) for x in feature["features"]]

multi_intersects = " OR ".join([f"ST_INTERSECTS(geom, (?))" for _ in areas])
# Execute the SQL query to 
query = f"""
SELECT *, ST_POINT(long, lat) as geom, datediff('hour', now() AT TIME ZONE 'UTC', timestamp)
FROM db.public.last_position
WHERE ({multi_intersects}) AND datediff('hour', now()::timestamp, timestamp) > -6
"""

con.execute(query, [area.wkt for area in areas]).df()